Notebook used to estimate Granger Causality between AINI variables and log returns.

In [2]:
import pandas as pd
import numpy as np
import sys
from pathlib import Path

# Get the project root: notebooks/AI_narrative_index
project_root = Path.cwd().parent

# Add src/scripts & modelling to the Python modules search path
sys.path.append(str(project_root / "src" / "scripts"))
sys.path.append(str(project_root / "src" / "modelling"))

# import custom functions
from granger_causality import  estimate_bootstraped_gc

Calculate Granger Causality and Transfer entropy for binary ( O / 1) AINI

load & verify data

In [4]:
 # get variables path
var_path = project_root / "data" / "processed" / "variables" 

# get financial data path
fin_path = project_root / "data" / "raw" / "financial" 

# load financial data
fin_data = pd.read_csv(fin_path / "full_daily_2023_2025.csv")

# load AINI predictions (0 to 1 index)
aini_data = pd.read_csv(var_path / "binary_AINI_variables.csv")


create log returns

In [3]:
# ensure sorting
fin_data['Date'] = pd.to_datetime(fin_data['Date'])
fin_data = fin_data.sort_values(['Ticker', 'Date'])

# Calculate log returns by Ticker
fin_data['LogReturn'] = fin_data.groupby('Ticker')['Adj Close'].transform(lambda x: np.log(x) - np.log(x.shift(1)))
fin_data = fin_data.dropna(subset=['LogReturn'])

# create distinct df of log_returns & ticker
ticker_dfs = {}
fin_data

,Date,Ticker,Open,High,Low,Close,Adj Close,Volume,LogReturn
3121,2023-01-04,AAPL,126.889999,128.660004,125.080002,126.360001,124.744125,89113600,0.010261
3122,2023-01-05,AAPL,127.129997,127.769997,124.760002,125.019997,123.421242,80962700,-0.010661
3123,2023-01-06,AAPL,126.010002,130.289993,124.889999,129.619995,127.962402,87754700,0.036133
3124,2023-01-09,AAPL,130.470001,133.410004,129.889999,130.149994,128.485657,70790800,0.004081
3125,2023-01-10,AAPL,130.259995,131.259995,128.119995,130.729996,129.058228,63896200,0.004446
...,...,...,...,...,...,...,...,...,...
9355,2025-06-24,TSM,214.389999,220.369995,213.880005,220.089996,220.089996,14814700,0.045406
9356,2025-06-25,TSM,220.800003,223.500000,220.800003,222.740005,222.740005,9168400,0.011969
9357,2025-06-26,TSM,224.979996,225.220001,222.699997,224.009995,224.009995,7982200,0.005685
9358,2025-06-27,TSM,224.300003,228.880005,224.029999,228.570007,228.570007,13133000,0.020152


In [4]:
# verify financial data
print(fin_data.head())
print(fin_data.isna().any())

# verify AINI data
print(aini_data.head())
print(aini_data.isna().any())

           Date Ticker        Open        High         Low       Close  \
3121 2023-01-04   AAPL  126.889999  128.660004  125.080002  126.360001   
3122 2023-01-05   AAPL  127.129997  127.769997  124.760002  125.019997   
3123 2023-01-06   AAPL  126.010002  130.289993  124.889999  129.619995   
3124 2023-01-09   AAPL  130.470001  133.410004  129.889999  130.149994   
3125 2023-01-10   AAPL  130.259995  131.259995  128.119995  130.729996   

       Adj Close    Volume  LogReturn  
3121  124.744125  89113600   0.010261  
3122  123.421242  80962700  -0.010661  
3123  127.962402  87754700   0.036133  
3124  128.485657  70790800   0.004081  
3125  129.058228  63896200   0.004446  
Date         False
Ticker       False
Open         False
High         False
Low          False
Close        False
Adj Close    False
Volume       False
LogReturn    False
dtype: bool
         date  normalized_AINI  simple_AINI  EMA_02  EMA_08  \
0  2023-04-01              0.0            0     0.0     0.0   
1  202

Create subsets for different periods

In [5]:
# Ensure both columns are datetime
fin_data['Date'] = pd.to_datetime(fin_data['Date'])
aini_data['date'] = pd.to_datetime(aini_data['date'])

# Define thresholds
threshold_23 = pd.Timestamp('2023-12-31')
threshold_24 = pd.Timestamp('2024-01-01')
threshold_25 = pd.Timestamp('2025-01-01')

# Filter data by year
fin_data_23 = fin_data[fin_data['Date'] < threshold_24]
fin_data_24 = fin_data[(fin_data['Date'] > threshold_23) & (fin_data['Date'] < threshold_25)]
fin_data_25 = fin_data[fin_data['Date'] >= threshold_25]

# overlapping
fin_data_23_24 = fin_data[fin_data['Date'] <= threshold_25]
fin_data_24_25 = fin_data[fin_data['Date'] > threshold_23]
fin_data_23_24

fin_data_by_year = {
    2023: fin_data_23,
    2024: fin_data_24,
    2025: fin_data_25,
    "2023_24": fin_data_23_24,
    "2024_25": fin_data_24_25,
    "2023_24_25": fin_data  
}

Estimate Grancger Causality and reverse Granger Causality between AINI variables & Stocks / ETFs

In [6]:
# estimate bootstrapped gc
gc_df_normal = estimate_bootstraped_gc(
    aini_data=aini_data,
    aini_variants=None,
    fin_data_by_year=fin_data_by_year,
    lag_range=range(1, 3),
    n_bootstrap=10000,
    alpha=0.05
    )

# estimate bootstrapped gc
gc_df_reverse = estimate_bootstraped_gc(
    aini_data=aini_data,
    aini_variants=None,
    fin_data_by_year=fin_data_by_year,
    lag_range=range(1, 3),
    n_bootstrap=10000,
    alpha=0.05,
    reverse = True
    )

gc_df_normal

  0%|          | 0/4 [00:00<?, ?it/s]c:\Users\PC\Desktop\Masterarbeit\AI_narrative_index\src\modelling\format_te_gc_inputs.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fin_data['Date'] = pd.to_datetime(fin_data['Date'])
c:\Users\PC\Desktop\Masterarbeit\AI_narrative_index\.venv\lib\site-packages\statsmodels\tsa\stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
c:\Users\PC\Desktop\Masterarbeit\AI_narrative_index\src\modelling\format_te_gc_inputs.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

Bootstrapped Granger results saved to: C:\Users\PC\Desktop\Masterarbeit\AI_narrative_index\data\processed\variables\gc_bootstrap_F_all_lags_groupwise_AINI_to_return_binary.csv


  0%|          | 0/4 [00:00<?, ?it/s]c:\Users\PC\Desktop\Masterarbeit\AI_narrative_index\src\modelling\format_te_gc_inputs.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fin_data['Date'] = pd.to_datetime(fin_data['Date'])
c:\Users\PC\Desktop\Masterarbeit\AI_narrative_index\.venv\lib\site-packages\statsmodels\tsa\stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
c:\Users\PC\Desktop\Masterarbeit\AI_narrative_index\src\modelling\format_te_gc_inputs.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

Bootstrapped Granger results saved to: C:\Users\PC\Desktop\Masterarbeit\AI_narrative_index\data\processed\variables\gc_bootstrap_F_all_lags_groupwise_return_to_AINI_binary.csv


,Ticker,AINI_variant,Year,Lag,Original_F,Original_F_pval,Empirical_F_pval,AIC,BIC,N_boot,...,White_pval,BP_reject,White_reject,Reverse,Window,coef_x1,coef_const,coef_x2,BH_reject_F,BH_corr_F_pval
0,AAPL,normalized_AINI,2023,1,0.051020,0.821549,0.827317,-1127.542110,-1121.079893,10000,...,0.348888,False,False,False,None,0.076196,0.000742,NaN,False,0.998000
1,AAPL,normalized_AINI,2023,2,0.231481,0.793592,0.793721,-1118.757204,-1109.079964,10000,...,0.651306,False,False,False,None,0.079275,0.000785,-0.024402,False,0.998000
2,AAPL,EMA_02,2023,1,0.000159,0.989959,0.991901,-1127.542110,-1121.079893,10000,...,0.348888,False,False,False,None,0.076196,0.000742,NaN,False,0.998000
3,AAPL,EMA_02,2023,2,0.002237,0.997766,0.998000,-1118.757204,-1109.079964,10000,...,0.651306,False,False,False,None,0.079275,0.000785,-0.024402,False,0.998000
4,AAPL,EMA_08,2023,1,0.053463,0.817401,0.819418,-1127.542110,-1121.079893,10000,...,0.348888,False,False,False,None,0.076196,0.000742,NaN,False,0.998000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,TSM,EMA_02,2024_25,2,0.244019,0.783609,0.781822,-1503.265558,-1491.726241,10000,...,0.757113,False,False,False,None,-0.099967,0.001797,0.014557,False,0.963304
716,TSM,EMA_08,2024_25,1,0.275203,0.600201,0.594941,-1510.135129,-1502.436479,10000,...,0.481627,False,False,False,None,-0.099666,0.001768,NaN,False,0.963304
717,TSM,EMA_08,2024_25,2,0.161728,0.850738,0.851815,-1503.265558,-1491.726241,10000,...,0.757113,False,False,False,None,-0.099967,0.001797,0.014557,False,0.963304
718,TSM,normalized_AINI_growth,2024_25,1,0.001815,0.966042,0.963304,-1510.135129,-1502.436479,10000,...,0.481627,False,False,False,None,-0.099666,0.001768,NaN,False,0.963304


Estimate Granger Causality for polarity AINI with windowsize (w) = 0

In [9]:
# load financial data
fin_data = pd.read_csv(fin_path / "full_daily_2023_2025.csv")

# load AINI predictions
aini_data_w0 = pd.read_csv(var_path / "w0_AINI_variables.csv")


In [10]:
# ensure sorting
fin_data['Date'] = pd.to_datetime(fin_data['Date'])
fin_data = fin_data.sort_values(['Ticker', 'Date'])

# Calculate log returns by Ticker
fin_data['LogReturn'] = fin_data.groupby('Ticker')['Adj Close'].transform(lambda x: np.log(x) - np.log(x.shift(1)))
fin_data = fin_data.dropna(subset=['LogReturn'])

# create distinct df of log_returns & ticker
ticker_dfs = {}

# Ensure both columns are datetime
fin_data['Date'] = pd.to_datetime(fin_data['Date'])
aini_data_w1['date'] = pd.to_datetime(aini_data_w1['date'])

# Define thresholds
threshold_23 = pd.Timestamp('2023-12-31')
threshold_24 = pd.Timestamp('2024-01-01')
threshold_25 = pd.Timestamp('2025-01-01')

# Filter data by year
fin_data_23 = fin_data[fin_data['Date'] < threshold_24]
fin_data_24 = fin_data[(fin_data['Date'] > threshold_23) & (fin_data['Date'] < threshold_25)]
fin_data_25 = fin_data[fin_data['Date'] >= threshold_25]

# overlapping
fin_data_23_24 = fin_data[fin_data['Date'] <= threshold_25]
fin_data_24_25 = fin_data[fin_data['Date'] > threshold_23]
fin_data_23_24

fin_data_by_year = {
    2023: fin_data_23,
    2024: fin_data_24,
    2025: fin_data_25,
    "2023_24": fin_data_23_24,
    "2024_25": fin_data_24_25,
    "2023_24_25": fin_data  
}



In [11]:
# estimate bootstrapped gc
gc_df_normal = estimate_bootstraped_gc(
    aini_data=aini_data_w0,
    aini_variants=None,
    fin_data_by_year=fin_data_by_year,
    lag_range=range(1, 3),
    n_bootstrap=10000,
    alpha=0.05,
    window = 0

    )

# estimate bootstrapped gc
gc_df_reverse = estimate_bootstraped_gc(
    aini_data=aini_data_w0,
    aini_variants=None,
    fin_data_by_year=fin_data_by_year,
    lag_range=range(1, 3),
    n_bootstrap=10000,
    alpha=0.05,
    reverse = True,
    window = 0
    )


  0%|          | 0/4 [00:00<?, ?it/s]c:\Users\PC\Desktop\Masterarbeit\AI_narrative_index\src\modelling\format_te_gc_inputs.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fin_data['Date'] = pd.to_datetime(fin_data['Date'])
c:\Users\PC\Desktop\Masterarbeit\AI_narrative_index\.venv\lib\site-packages\statsmodels\tsa\stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
c:\Users\PC\Desktop\Masterarbeit\AI_narrative_index\src\modelling\format_te_gc_inputs.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

Bootstrapped Granger results saved to: C:\Users\PC\Desktop\Masterarbeit\AI_narrative_index\data\processed\variables\gc_bootstrap_F_all_lags_groupwise_AINI_to_return_3class_w0.csv


  0%|          | 0/4 [00:00<?, ?it/s]c:\Users\PC\Desktop\Masterarbeit\AI_narrative_index\src\modelling\format_te_gc_inputs.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fin_data['Date'] = pd.to_datetime(fin_data['Date'])
c:\Users\PC\Desktop\Masterarbeit\AI_narrative_index\.venv\lib\site-packages\statsmodels\tsa\stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
c:\Users\PC\Desktop\Masterarbeit\AI_narrative_index\src\modelling\format_te_gc_inputs.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

Bootstrapped Granger results saved to: C:\Users\PC\Desktop\Masterarbeit\AI_narrative_index\data\processed\variables\gc_bootstrap_F_all_lags_groupwise_return_to_AINI_3class_w0.csv


Estimate Granger Causality for polarity AINI with windowsize (w) = 1

In [7]:
# load financial data
fin_data = pd.read_csv(fin_path / "full_daily_2023_2025.csv")

# load AINI predictions
aini_data_w1 = pd.read_csv(var_path / "w1_AINI_variables.csv")


In [8]:
# ensure sorting
fin_data['Date'] = pd.to_datetime(fin_data['Date'])
fin_data = fin_data.sort_values(['Ticker', 'Date'])

# Calculate log returns by Ticker
fin_data['LogReturn'] = fin_data.groupby('Ticker')['Adj Close'].transform(lambda x: np.log(x) - np.log(x.shift(1)))
fin_data = fin_data.dropna(subset=['LogReturn'])

# create distinct df of log_returns & ticker
ticker_dfs = {}

# Ensure both columns are datetime
fin_data['Date'] = pd.to_datetime(fin_data['Date'])
aini_data_w1['date'] = pd.to_datetime(aini_data_w1['date'])

# Define thresholds
threshold_23 = pd.Timestamp('2023-12-31')
threshold_24 = pd.Timestamp('2024-01-01')
threshold_25 = pd.Timestamp('2025-01-01')

# Filter data by year
fin_data_23 = fin_data[fin_data['Date'] < threshold_24]
fin_data_24 = fin_data[(fin_data['Date'] > threshold_23) & (fin_data['Date'] < threshold_25)]
fin_data_25 = fin_data[fin_data['Date'] >= threshold_25]

# overlapping
fin_data_23_24 = fin_data[fin_data['Date'] <= threshold_25]
fin_data_24_25 = fin_data[fin_data['Date'] > threshold_23]
fin_data_23_24

fin_data_by_year = {
    2023: fin_data_23,
    2024: fin_data_24,
    2025: fin_data_25,
    "2023_24": fin_data_23_24,
    "2024_25": fin_data_24_25,
    "2023_24_25": fin_data  
}



In [9]:
# estimate bootstrapped gc
gc_df_normal = estimate_bootstraped_gc(
    aini_data=aini_data_w1,
    aini_variants=None,
    fin_data_by_year=fin_data_by_year,
    lag_range=range(1, 3),
    n_bootstrap=10000,
    alpha=0.05,
    window = 1

    )

# estimate bootstrapped gc
gc_df_reverse = estimate_bootstraped_gc(
    aini_data=aini_data_w1,
    aini_variants=None,
    fin_data_by_year=fin_data_by_year,
    lag_range=range(1, 3),
    n_bootstrap=10000,
    alpha=0.05,
    reverse = True,
    window = 1
    )


  0%|          | 0/4 [00:00<?, ?it/s]c:\Users\PC\Desktop\Masterarbeit\AI_narrative_index\src\modelling\format_te_gc_inputs.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fin_data['Date'] = pd.to_datetime(fin_data['Date'])
c:\Users\PC\Desktop\Masterarbeit\AI_narrative_index\.venv\lib\site-packages\statsmodels\tsa\stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
c:\Users\PC\Desktop\Masterarbeit\AI_narrative_index\src\modelling\format_te_gc_inputs.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

Bootstrapped Granger results saved to: C:\Users\PC\Desktop\Masterarbeit\AI_narrative_index\data\processed\variables\gc_bootstrap_F_all_lags_groupwise_AINI_to_return_3class_w1.csv


  0%|          | 0/4 [00:00<?, ?it/s]c:\Users\PC\Desktop\Masterarbeit\AI_narrative_index\src\modelling\format_te_gc_inputs.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fin_data['Date'] = pd.to_datetime(fin_data['Date'])
c:\Users\PC\Desktop\Masterarbeit\AI_narrative_index\.venv\lib\site-packages\statsmodels\tsa\stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
c:\Users\PC\Desktop\Masterarbeit\AI_narrative_index\src\modelling\format_te_gc_inputs.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

Bootstrapped Granger results saved to: C:\Users\PC\Desktop\Masterarbeit\AI_narrative_index\data\processed\variables\gc_bootstrap_F_all_lags_groupwise_return_to_AINI_3class_w1.csv


Estimate Granger Causality for polarity AINI with windowsize (w) = 2

In [2]:
# Get notebook's parent file location 
project_root = Path().resolve().parents[0] 

# get reports path for figures (see below)
fig_path = project_root / "reports" / "figures" 

# get variables path
var_path = project_root / "data" / "processed" / "variables" 

# get financial data path
fin_path = project_root / "data" / "raw" / "financial" 

# load financial data
fin_data = pd.read_csv(fin_path / "full_daily_2023_2025.csv")

# load AINI predictions
aini_data_w2 = pd.read_csv(var_path / "w2_AINI_variables.csv")

In [3]:
# ensure sorting
fin_data['Date'] = pd.to_datetime(fin_data['Date'])
fin_data = fin_data.sort_values(['Ticker', 'Date'])

# Calculate log returns by Ticker
fin_data['LogReturn'] = fin_data.groupby('Ticker')['Adj Close'].transform(lambda x: np.log(x) - np.log(x.shift(1)))
fin_data = fin_data.dropna(subset=['LogReturn'])

# create distinct df of log_returns & ticker
ticker_dfs = {}

# Ensure both columns are datetime
fin_data['Date'] = pd.to_datetime(fin_data['Date'])
aini_data_w2['date'] = pd.to_datetime(aini_data_w2['date'])

# Define thresholds
threshold_23 = pd.Timestamp('2023-12-31')
threshold_24 = pd.Timestamp('2024-01-01')
threshold_25 = pd.Timestamp('2025-01-01')

# Filter data by year
fin_data_23 = fin_data[fin_data['Date'] < threshold_24]
fin_data_24 = fin_data[(fin_data['Date'] > threshold_23) & (fin_data['Date'] < threshold_25)]
fin_data_25 = fin_data[fin_data['Date'] >= threshold_25]

# overlapping
fin_data_23_24 = fin_data[fin_data['Date'] <= threshold_25]
fin_data_24_25 = fin_data[fin_data['Date'] > threshold_23]
fin_data_23_24

fin_data_by_year = {
    2023: fin_data_23,
    2024: fin_data_24,
    2025: fin_data_25,
    "2023_24": fin_data_23_24,
    "2024_25": fin_data_24_25,
    "2023_24_25": fin_data  
}


In [4]:
# estimate bootstrapped gc
gc_df_normal = estimate_bootstraped_gc(
    aini_data=aini_data_w2,
    aini_variants=None,
    fin_data_by_year=fin_data_by_year,
    lag_range=range(1, 3),
    n_bootstrap=10000,
    alpha=0.05,
    window = 2

    )

# estimate bootstrapped gc
gc_df_reverse = estimate_bootstraped_gc(
    aini_data=aini_data_w2,
    aini_variants=None,
    fin_data_by_year=fin_data_by_year,
    lag_range=range(1, 3),
    n_bootstrap=10000,
    alpha=0.05,
    reverse = True,
    window = 2
    )
gc_df_normal

  0%|          | 0/4 [00:00<?, ?it/s]c:\Users\PC\Desktop\Masterarbeit\AI_narrative_index\src\modelling\format_te_gc_inputs.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fin_data['Date'] = pd.to_datetime(fin_data['Date'])
c:\Users\PC\Desktop\Masterarbeit\AI_narrative_index\.venv\lib\site-packages\statsmodels\tsa\stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
c:\Users\PC\Desktop\Masterarbeit\AI_narrative_index\src\modelling\format_te_gc_inputs.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

Bootstrapped Granger results saved to: C:\Users\PC\Desktop\Masterarbeit\AI_narrative_index\data\processed\variables\gc_bootstrap_F_all_lags_groupwise_AINI_to_return_3class_w2.csv


  0%|          | 0/4 [00:00<?, ?it/s]c:\Users\PC\Desktop\Masterarbeit\AI_narrative_index\src\modelling\format_te_gc_inputs.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fin_data['Date'] = pd.to_datetime(fin_data['Date'])
c:\Users\PC\Desktop\Masterarbeit\AI_narrative_index\.venv\lib\site-packages\statsmodels\tsa\stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
c:\Users\PC\Desktop\Masterarbeit\AI_narrative_index\src\modelling\format_te_gc_inputs.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

Bootstrapped Granger results saved to: C:\Users\PC\Desktop\Masterarbeit\AI_narrative_index\data\processed\variables\gc_bootstrap_F_all_lags_groupwise_return_to_AINI_3class_w2.csv


,Ticker,AINI_variant,Year,Lag,Original_F,Original_F_pval,Empirical_F_pval,AIC,BIC,N_boot,...,White_pval,BP_reject,White_reject,Reverse,Window,coef_x1,coef_const,coef_x2,BH_reject_F,BH_corr_F_pval
0,AAPL,normalized_AINI,2023,1,0.125150,0.723919,0.727127,-1127.542110,-1121.079893,10000,...,0.348888,False,False,False,2,0.076196,0.000742,NaN,False,0.979602
1,AAPL,normalized_AINI,2023,2,0.089153,0.914745,0.913309,-1118.757204,-1109.079964,10000,...,0.651306,False,False,False,2,0.079275,0.000785,-0.024402,False,0.979602
2,AAPL,EMA_02,2023,1,0.000618,0.980194,0.979602,-1127.542110,-1121.079893,10000,...,0.348888,False,False,False,2,0.076196,0.000742,NaN,False,0.979602
3,AAPL,EMA_02,2023,2,0.067823,0.934450,0.932007,-1118.757204,-1109.079964,10000,...,0.651306,False,False,False,2,0.079275,0.000785,-0.024402,False,0.979602
4,AAPL,EMA_08,2023,1,0.160407,0.689246,0.684132,-1127.542110,-1121.079893,10000,...,0.348888,False,False,False,2,0.076196,0.000742,NaN,False,0.979602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,TSM,EMA_02,2024_25,2,0.191529,0.825785,0.819218,-1503.265558,-1491.726241,10000,...,0.757113,False,False,False,2,-0.099967,0.001797,0.014557,False,0.819218
716,TSM,EMA_08,2024_25,1,0.185432,0.667015,0.659034,-1510.135129,-1502.436479,10000,...,0.481627,False,False,False,2,-0.099666,0.001768,NaN,False,0.819218
717,TSM,EMA_08,2024_25,2,0.421616,0.656328,0.642436,-1503.265558,-1491.726241,10000,...,0.757113,False,False,False,2,-0.099967,0.001797,0.014557,False,0.819218
718,TSM,normalized_AINI_growth,2024_25,1,0.460139,0.498015,0.493751,-1510.135129,-1502.436479,10000,...,0.481627,False,False,False,2,-0.099666,0.001768,NaN,False,0.819218
